# Segmenting and Clustering Neighborhoods in Toronto

### Introduction
##### Web scraping the following Wikipedia page, 
###### https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M,
##### in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

Before we start,let's download all the dependencies that we will need.

In [112]:
#pip install requests BeautifulSoup4
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes
import requests

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

from selenium import webdriver
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


In [115]:
results = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
#print(results.status_code)
#print(results.headers)
src=results.content
soup = BeautifulSoup(src,'lxml')
table = soup.find("table",attrs={"class":"wikitable"})
#table_header = table.find_all("th")
table_body = table.find_all("td")
print(table_header)
print(table_body[3])

[<th>Postal Code
</th>, <th>Borough
</th>, <th>Neighborhood
</th>]
<td>M2A
</td>


In [116]:
pc=[]
bor=[]
nb=[]
for i in range(0,len(table_body),3):
    pc.append((table_body[i].text).strip())
    bor.append((table_body[i+1].text).strip())
    nb.append((table_body[i+2].text).strip())
    #print(table_body[i].text)
#print(pc)
#print(bor)
#print(nb)

Toronto = (pd.DataFrame({'PostalCode':pc,'Borough':bor,'Neighborhood':nb})).reset_index(drop=True)
Toronto.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


**Removing Not Assigned Boroughs**

In [129]:
Toronto = Toronto[Toronto.Borough != 'Not assigned']
Toronto.head()
Toronto.describe(include='all')

,PostalCode,Borough,Neighborhood
count,103,103,103
unique,103,10,99
top,M2P,North York,Downsview
freq,1,24,4


**Setting Not Assigned Neighborhoods to same Borough**

In [ ]:
Toronto['Neighborhood'] = np.where(Toronto['Neighborhood'] == 'Not assigned', Toronto['Borough'], Toronto['Neighborhood'])
Toronto.describe(include='all')

**Created DataFrame Shape**

In [137]:
Toronto.shape

(103, 3)

In [ ]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(Toronto['PostalCode']))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
print('finished getting longitude and lattitude')

In [ ]:
Toronto.head()